In [29]:
# 1-qadam. kutubxonalar chaqiramiz:
import os
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from tqdm import tqdm
from ultralytics import YOLO
import torch
import cv2


In [4]:
# 2-qadam. Asosiy dataset papkasini yaratamiz:
dataset_folder = "dataset"
subfolders = ["images/train", "images/val", "labels/train", "labels/val"]
for sub in subfolders:
    path = os.path.join(dataset_folder, sub)
    os.makedirs(path, exist_ok = True)
print("Bosh dataset papkalar yaratildi.")


Bosh dataset papkalar yaratildi.


In [7]:
# 3. Rasm fayllarni train/val papkalarga ajratish 
images_folders = "images"    # original rasmlar papkasi
dataset_images_folder = "dataset/images"     # YOLO dataset images papkasi
images_files = [f for f in os.listdir(images_folders) if f.endswith(".png")]

# Train / Val split (80/20)
train_files,val_files = train_test_split(images_files, train_size=0.8, random_state =42)

# fayllarni nusxalaymiz:
for f  in train_files:
    shutil.copy(os.path.join(images_folders,f), os.path.join(dataset_images_folder,"train", f))
for f in val_files:
    shutil.copy(os.path.join(images_folders,f), os.path.join(dataset_images_folder, "val",f))
print(f"Rasm fayllar \ntrain: {len(train_files)} \nval: {len(val_files)} \npapkalarga nusxalandi.")

Rasm fayllar 
train: 4000 
val: 1000 
papkalarga nusxalandi.


In [4]:
# 4. XML fayllarni YOLO formatga konvertatsiya qilish
import os
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from tqdm import tqdm


xml_folder = "annotations"   # original xml fayllar papkasi
dataset_labels = "dataset/labels"  # yolo format labels papkasi
train_ratio = 0.8             # train/val split foizi

# class mapping:
class_dict = {"helmet":0, "head":1}

#xml fayllar royhati:
xml_files = [f for f in os.listdir(xml_folder) if f.endswith(".xml")]

#train/val ga split qilamiz:
train_files, val_files = train_test_split(xml_files, train_size = train_ratio, random_state = 42)
def convert_xml_to_yolo(xml_file, save_dir):
    tree = ET.parse(os.path.join(xml_folder, xml_file))
    root = tree.getroot()
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)
    yolo_lines = []
    for obj in root.findall("object"):
        cls_name = obj.find('name').text
        if cls_name not in class_dict:
            continue
        cls_id = class_dict[cls_name]
        bndbox = obj.find("bndbox")      #Obyektning bounding box bo‘limini topadi.  <xmin>, <ymin>, <xmax>, <ymax> taglari shu yerda bo‘ladi.
        xmin = float(bndbox.find("xmin").text)    #Har bir qiymatni matn → son ga o‘giradi.
        ymin = float(bndbox.find("ymin").text)
        xmax = float(bndbox.find("xmax").text)
        ymax = float(bndbox.find("ymax").text)

        # Clipping: rasm chegarasiga moslash
        xmin = max(0, min(xmin, width))   #Bu qatorlar bounding box koordinatalarini rasm chegarasidan chiqib ketmasligi uchun tuzatadi.
        ymin = max(0, min(ymin, height))
        xmax = max(0, min(xmax, width))
        ymax = max(0, min(ymax, height))

        # YOLO formatga o'zgartirish     Bu qatorlar YOLO talab qiladigan 5 ta qiymatni hisoblaydi:  class x_center y_center width height
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        w = (xmax - xmin) / width
        h = (ymax - ymin) / height
        # Out-of-bounds tekshirish
        if x_center < 0 or x_center > 1 or y_center < 0 or y_center > 1 or w <= 0 or h <= 0:
            print(f" Out of bounds: {xml_file} ({cls_name})")
            continue
        
        yolo_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")
    
    # TXT faylni saqlash     Agar XML faylda obyektlar bo‘lsa, ular YOLO formatga o‘tkaziladi va rasm nomiga mos .txt fayl ichiga yoziladi.
    if yolo_lines:
        txt_filename = os.path.splitext(xml_file)[0] + ".txt"
        with open(os.path.join(save_dir, txt_filename), "w") as f:
            f.write("\n".join(yolo_lines))
# --- Train fayllarni konvert qilish ---    train fayllarni YOLO txt ga aylantiradi → dataset/labels/train/ ga saqlaydi.
for xml_file in tqdm(train_files, desc="Converting Train XML"):
    convert_xml_to_yolo(xml_file, os.path.join(dataset_labels, "train"))

# --- Val fayllarni konvert qilish ---   val fayllarni YOLO txt ga aylantiradi → dataset/labels/val/ ga saqlaydi.
for xml_file in tqdm(val_files, desc="Converting Val XML"):
    convert_xml_to_yolo(xml_file, os.path.join(dataset_labels, "val"))

print(" XML fayllar YOLO formatga muvaffaqiyatli o‘tkazildi va train/val papkalarga joylandi!")

Converting Val XML: 100%|████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2618.39it/s]

 XML fayllar YOLO formatga muvaffaqiyatli o‘tkazildi va train/val papkalarga joylandi!


In [9]:
# 5. YAML file yaratamiz:
import yaml

# YAML fayl mazmuni     Bu kod YOLO uchun kerak bo‘lgan dataset.yaml faylni yaratadi va YOLO model training vaqtida rasm va label qayerda turganini biladi.
data_yaml = {
    'path': 'dataset',         # Dataset asosiy papkasi
    'train': 'images/train',   # Train rasmlar
    'val': 'images/val',       # Validation rasmlar
    'names': ['helmet', 'head']  # Klasslar
}

# YAML faylni saqlash
with open('dataset.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

print("✅ dataset.yaml fayli yaratildi!")

✅ dataset.yaml fayli yaratildi!


In [7]:
!pip install -U ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [10]:
# 6. GPU tekshirish
from ultralytics import YOLO    #Bu kod YOLO va PyTorch to‘g‘ri o‘rnatilganini va GPU ishlash imkoniyatini tekshiradi.
import torch

print("YOLO imported successfully")
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


YOLO imported successfully
Torch version: 2.5.1+cu121
CUDA available: True


In [11]:
# YOLO train boshlaymiz:
# Import
from ultralytics import YOLO

# Modelni yuklash (pretrained YOLOv8n)
model = YOLO("yolov8n.pt")

# Train qilish
model.train(
    data="dataset.yaml",      # dataset fayl manzili
    epochs=50,                # epochlar soni
    imgsz=640,                # rasm o'lchami
    batch=8,                  # batch size, GPU ga qarab o'zgartiring
    project="runs/detect",    # natijalar saqlanadigan folder
    name="helmet_train",      # eksperiment nomi
    exist_ok=True             # agar folder bo'lsa ustiga yozadi
)


Ultralytics 8.3.235  Python-3.11.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=helmet_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022977D27F50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [14]:
import os
os.getcwd()


'C:\\Users\\bahti\\computer_vision\\LOYIHA\\loyiha-1'

# 8-Test qilamiz:

In [18]:
# 8-Test qilamiz :
from ultralytics import YOLO

model_path = r"C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\runs\detect\helmet_train\weights\best.pt"

model = YOLO(model_path)

print("Model yuklandi:", model_path)


Model yuklandi: C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\runs\detect\helmet_train\weights\best.pt


In [19]:
# rasmni test qilish
model.predict(r"C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\test_uchun\rasm1.png", save=True)
model.predict(
    source="test_uchun/rasm1.png",
    save=True,
    project=r"C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\Natijalar",
    name="test1",
    exist_ok=True
)



image 1/1 C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\test_uchun\rasm1.png: 640x640 3 helmets, 23.6ms
Speed: 5.0ms preprocess, 23.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\runs\detect\predict2

image 1/1 C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\test_uchun\rasm1.png: 640x640 3 helmets, 23.6ms
Speed: 3.8ms preprocess, 23.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\Natijalar\test1


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'helmet', 1: 'head'}
 obb: None
 orig_img: array([[[ 83,  80,  79],
         [ 83,  80,  79],
         [ 74,  78,  80],
         ...,
         [169, 165, 163],
         [158, 160, 162],
         [162, 161, 161]],
 
        [[ 79,  83,  86],
         [ 83,  80,  79],
         [ 83,  80,  79],
         ...,
         [169, 165, 163],
         [162, 161, 161],
         [162, 161, 161]],
 
        [[ 79,  83,  86],
         [ 83,  80,  79],
         [ 79,  83,  86],
         ...,
         [164, 166, 168],
         [162, 161, 161],
         [162, 161, 161]],
 
        ...,
 
        [[249, 248, 249],
         [249, 248, 249],
         [249, 248, 249],
         ...,
         [255, 234, 168],
         [255, 234, 168],
         [255, 234, 168]],
 
        [[249, 248, 249],
         [249, 248, 249],
         [249, 248, 249],
         ...,
       

In [ ]:
# video test qilish:
# model yuklaymiz:
from ultralytics import YOLO
model = YOLO("C:/Users/bahti/computer_vision/LOYIHA/loyiha-1/runs/detect/helmet_train/weights/best.pt")

model.predict(r"C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\test_uchun\video1.mp4", save=True)
model.predict(
    source="test_uchun/video1.mp4",
    save=True,
    project=r"C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\Natijalar",
    name="test2",
    exist_ok=True
)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/7353) C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\test_uchun\video1.mp4: 384x640 (no detections), 19.0ms
video 1/1 (frame 2/7353) C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\test_uchun\video1.mp4: 384x640 (no detections), 18.0ms
video 1/1 (frame 3/7353) C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\test_uchun\video1.mp4: 384x640 1 head, 16.3ms
video 1/1 (frame 4/7353) C:\Users\bahti\computer_vision\LOYIHA\loyiha-1\test_uchun\vide